In [ ]:
# win+R --> 跳到路径所在磁盘 g:
# tensorboard --logdir=G:\Anaconda3\My_Jupyter\logdir
# http://PC-20181031UPGR:6006 刷新

In [ ]:
# 单击右键可以把每个模块移出来 写论文时可以用

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector # 没用到 这是哪个进三维图的
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 每批次训练数据大小 和 批次大小
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

# 参数概要 （没有用到）
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)             # 标准差
        tf.summary.scalar('max',tf.reduce_max(var))    # 最大值
        tf.summary.scalar('min',tf.reduce_min(var))    # 最小值
        tf.summary.histogram('histogram',var)          # 直方图

# 初始化权值
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev = 0.1) # 生成一个截断的正态分布
    return tf.Variable(initial,name=name)

# 初始化偏置
def bias_variable(shape,name):
    initial = tf.constant(0.1,shape = shape) # 生成一个截断的正态分布
    return tf.Variable(initial,name=name)

# 卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

# 池化层
def max_pooling_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# 命名空间

with tf.name_scope('input'):
    # 给输入数据预留位置
    x = tf.placeholder(tf.float32, shape = [None,784], name = 'x-input')
    y = tf.placeholder(tf.float32, shape = [None,10], name = 'y-input')
    
    with tf.name_scope('x_image'):
        # 改变x的格式转换为4D的向量 [batch,in_height,in_weight,in_channels]
        x_image = tf.reshape(x,[-1,28,28,1], name = 'x_image')
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder(tf.float32,name = 'keep_prob')    # 神经元的输出概率

with tf.name_scope('Conv1'):
    with tf.name_scope('W_conv1'):
        # 初始化第一个卷积层的权重和偏置
        W_conv1 = weight_variable([5,5,1,32], name='W_conv1')# 卷积是5×5的采样窗口 1是x_image的in_channels是1维灰度图像 32是使用32个卷积核得到32个特征图
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32], name='b_conv1')                                                    # 每一个卷积核只需要一个偏置值 32个卷积核就是32个偏置值
    with tf.name_scope('W_conv_x_plus_b1'):
        W_conv_x_plus_b1 = conv2d(x_image,W_conv1) + b_conv1
    with tf.name_scope('relu'):   
        # 把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数        
        h_conv1 = tf.nn.relu(W_conv_x_plus_b1)                                  # 经过卷积操作再加上偏置值 再激活 得到第一个卷积层的结果
    with tf.name_scope('max_pooling'):   
        h_pool1 = max_pooling_2x2(h_conv1)                                                           #进行max-pooling  得到第一个池化结果

with tf.name_scope('Conv2'):
    with tf.name_scope('W_conv2'):    
        # 初始化第二个卷积层的权重和偏置
        W_conv2 = weight_variable([5,5,32,64], name = 'W_conv2')                                    # 5×5的采样窗口 传入32个特征图 用64个卷积核得到 输出64个特征图
    with tf.name_scope('b_conv2'):        
        b_conv2 = bias_variable([64], name = 'b_conv2')
    with tf.name_scope('W_conv_x_plus_b1'):
        W_conv_x_plus_b2 = conv2d(h_pool1,W_conv2) + b_conv2       
    with tf.name_scope('relu'):          
        # 把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
        h_conv2 = tf.nn.relu(W_conv_x_plus_b2)
    with tf.name_scope('max_pooling'):         
        h_pool2 = max_pooling_2x2(h_conv2)                                                                   #进行max-pooling
    with tf.name_scope('h_pool2_flat'):
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64],name = 'h_pool2_flat')               # 把池化层2的输出扁平化为1维 是转置吗？
        
with tf.name_scope('Full_Connection_1'):
    with tf.name_scope('W_Full_Connection_1'):  
        W_fc1 = weight_variable([7*7*64,1024],name = 'W_Full_Connection_1')# 把上面那个平面展开 要7*7*64个神经元 全连接层一共有1024个神经元
    with tf.name_scope('b_Full_Connection_1'):  
        b_fc1 = bias_variable([1024],name = 'b_Full_Connection_1')           # 1024个偏置
    with tf.name_scope('Wx_plus_b1'):
        Wx_plus_b1 = tf.matmul(h_pool2_flat,W_fc1) + b_fc1
    with tf.name_scope('relu'):  
        h_fc1 = tf.nn.relu(Wx_plus_b1)                         # 求第一个全连接层的输出
    with tf.name_scope('h_fc1_drop'):
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob,name = 'h_fc1_drop')

with tf.name_scope('Full_Connection_2'):
    with tf.name_scope('W_Full_Connection_2'):  
        W_fc2 = weight_variable([1024,10],name = 'W_Full_Connection_2')  # 把上面那个平面展开 要7*7*64个神经元 全连接层一共有1024个神经元
    with tf.name_scope('b_Full_Connection_2'):  
        b_fc2 = bias_variable([10],name = 'b_Full_Connection_2')           # 1024个偏置
    with tf.name_scope('Wx_plus_b1'):
        Wx_plus_b2 = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
    with tf.name_scope('softmax'):      
        prediction = tf.nn.softmax(Wx_plus_b2)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = prediction),name = 'loss')
    tf.summary.scalar('loss',loss)  # 数据比较多的时候 才需要调用 variable_summaries(var)这个函数 这里 loss只有一个值

with tf.name_scope('train'):
    train = tf.train.AdamOptimizer(0.0001).minimize(loss)


with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))# 返回Ture False 求出最大的数在的位置看它俩返回的位置一样不
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    
        tf.summary.scalar('accuracy',accuracy)      # 算一下准确率的变化

merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('logdir/train',sess.graph)
    test_writer = tf.summary.FileWriter('logdir/test',sess.graph)
    for i in range(10001):
        #训练模型
        batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
        sess.run(train,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        #记录训练集计算的参数     # 准确率和交叉熵
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        train_writer.add_summary(summary,i)
        #记录测试集计算的参数
        batch_xs,batch_ys =  mnist.test.next_batch(batch_size)
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        test_writer.add_summary(summary,i)
 
        if i%1000==0:
            test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images[:6000],y:mnist.test.labels[:6000],keep_prob:1.0})
            train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images[:6000],y:mnist.train.labels[:6000],keep_prob:1.0})
            print('第' + str(i) + '次，train准确率 ' + str(train_acc) + ' test准确率 ' + str(test_acc))

tf.logging.set_verbosity(old_v)


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
# 第0次，train准确率 0.111666664 test准确率 0.10783333
# 第1000次，train准确率 0.967 test准确率 0.9701667
# 第2000次，train准确率 0.9803333 test准确率 0.981
# 第3000次，train准确率 0.98716664 test准确率 0.98483336
# 第4000次，train准确率 0.98983335 test准确率 0.9863333
# 第5000次，train准确率 0.9928333 test准确率 0.9875
# 第6000次，train准确率 0.9928333 test准确率 0.988
# 第7000次，train准确率 0.994 test准确率 0.98933333
# 第8000次，train准确率 0.9945 test准确率 0.99266666
# 第9000次，train准确率 0.996 test准确率 0.98983335
# 第10000次，train准确率 0.9955 test准确率 0.99

第0次，train准确率 0.111666664 test准确率 0.10783333
第1000次，train准确率 0.967 test准确率 0.9701667
第2000次，train准确率 0.9803333 test准确率 0.981
第3000次，train准确率 0.98716664 test准确率 0.98483336
第4000次，train准确率 0.98983335 test准确率 0.9863333
第5000次，train准确率 0.9928333 test准确率 0.9875
第6000次，train准确率 0.9928333 test准确率 0.988
第7000次，train准确率 0.994 test准确率 0.98933333
第8000次，train准确率 0.9945 test准确率 0.99266666
第9000次，train准确率 0.996 test准确率 0.98983335
第10000次，train准确率 0.9955 test准确率 0.99
